In [12]:
from bs4 import BeautifulSoup  
import requests
from pprint import pprint
import ftfy

In [13]:
#expected ünicode
print(ftfy.fix_text('uÌˆnicode'))

ünicode


In [3]:
#prePage is a soup object 
def nextPageUrl(prePage):
#   resp = requests.get(prePageUrl)
#   prePage = BeautifulSoup(resp.text, "html.parser")
    li = prePage.find("li", {"class": "next sprite"})
    a = li.find('a')
    nextPageUrl = a.attrs['href']
    #print(nextPageUrl)
    return nextPageUrl


In [4]:
def newsUrls(pageUrl):
    resp = requests.get(pageUrl)
    #print("request_code:", resp.status_code)
    soup = BeautifulSoup(resp.text, "html.parser")
    urls = []
    for h in soup.find_all("div", {"class": "searchResultsBox"}):
        a = h.find('a')
        urls.append(a.attrs['href'])
       
    return soup, urls

In [5]:
mainUrl = 'https://www.bloomberght.com/tum-ekonomi-haberleri'
resp = requests.get(mainUrl)
print(resp.status_code)
soup = BeautifulSoup(resp.text, "html.parser")


200


In [6]:
urls = []
for h in soup.find_all("div", {"class": "searchResultsBox"}):
    a = h.find('a')
    urls.append(a.attrs['href'])

urls

['/abd-de-issizlik-maasi-basvurulari-beklenenden-yuksek-cikti-2230648',
 '/kuresel-piyasalar-hisseler-cin-in-misilleme-aciklamasi-ile-dustu-tahviller-yukseldi-2230640',
 '/spot-piyasada-elektrik-fiyatlari-15082019-2230642',
 '/sigarada-otv-duzenlemesi-zam-olarak-doner-mi-2230590',
 '/borsa-gunun-ilk-yarisinda-geriledi-2230637',
 '/negatif-faizli-tahvil-toplami-16-trilyon-dolarla-rekor-kirdi-2230633',
 '/cin-abd-10-yeni-tarife-ile-trump-xi-anlasmasini-ihlal-ediyor-2230634',
 '/ekonomi-basininda-bugun-15-agustos-2019-2230628',
 '/ingiltere-de-perakende-satislar-yuzde-02-yukseldi-2230629',
 '/tcmb-repo-ihalesiyle-piyasaya-3-milyar-lira-verdi-2230630']

In [7]:
pageUrl = mainUrl
allNews = []

##5 pages scanned
for counter in range(3):
    prePage, news = newsUrls(pageUrl)
    for each in news:
        print('https://www.bloomberght.com' + each)
    allNews += news
    pageUrl = nextPageUrl(prePage)
    #for each in news:
        #print(each)
    #print(".", end="")
    

https://www.bloomberght.com/abd-de-issizlik-maasi-basvurulari-beklenenden-yuksek-cikti-2230648
https://www.bloomberght.com/kuresel-piyasalar-hisseler-cin-in-misilleme-aciklamasi-ile-dustu-tahviller-yukseldi-2230640
https://www.bloomberght.com/spot-piyasada-elektrik-fiyatlari-15082019-2230642
https://www.bloomberght.com/sigarada-otv-duzenlemesi-zam-olarak-doner-mi-2230590
https://www.bloomberght.com/borsa-gunun-ilk-yarisinda-geriledi-2230637
https://www.bloomberght.com/negatif-faizli-tahvil-toplami-16-trilyon-dolarla-rekor-kirdi-2230633
https://www.bloomberght.com/cin-abd-10-yeni-tarife-ile-trump-xi-anlasmasini-ihlal-ediyor-2230634
https://www.bloomberght.com/ekonomi-basininda-bugun-15-agustos-2019-2230628
https://www.bloomberght.com/ingiltere-de-perakende-satislar-yuzde-02-yukseldi-2230629
https://www.bloomberght.com/tcmb-repo-ihalesiyle-piyasaya-3-milyar-lira-verdi-2230630
https://www.bloomberght.com/cavusoglu-guvenli-bolge-abd-nin-oyalama-taktikleri-gecerli-olmayacaktir-2230627
https

In [8]:
#for each in allNews:
#    print(each)

In [9]:
len(allNews)

30

In [10]:
def getRawContent(newsUrl):
    res = requests.get("https://www.bloomberght.com"+ newsUrl)
    soup = BeautifulSoup(res.text, "html.parser") 

    raw_content = soup.findAll("p")
    
    return raw_content


In [11]:
def getText(raw_content):
    
    document = []
    
    for par in raw_content:
        if par.text != "" and not par.find("a") and len(par.text) > 3:
            document.append(par.text)
    
    return document

In [14]:
def writeText(start,stop):
    for url in allNews[start:stop]:
        doc = getText(getRawContent(url))
        docs.append(doc)
        print(".", end="")

In [15]:
def eraseText(untill):
    curr_size = len(docs)
    if curr_size - untill <= 0:
        return
    for idx in range(curr_size - untill):
        docs.pop()

In [16]:
def isDuplicateExist(test_list):
    
    if len(test_list) != len(set(test_list)):
        print("WARNING!!!! \n There are duplicates in your list")
        return True
    else:
        print("All elements are distinct")
        return False
    

In [17]:
#def reinterpret_latin1_as_utf8(wrongtext):
#    newbytes = wrongtext.encode('latin-1', 'replace')
#    return newbytes.decode('utf-8', 'replace')
#

In [18]:
def writeToDisk(docs):
    f = open("TurkishDataSet.txt", "a")
    for i in range(len(docs)):
        str_doc = ' '.join(docs[i])
        str_doc += "\n"
        f.write(str_doc)
    
    f.close() 
        
        

In [26]:
docs = []

In [27]:
for url in allNews:
    rawContent = getRawContent(url)
    rawtext = getText(rawContent)
    rawtext = ' '.join(rawtext)
    text = ftfy.fix_text(rawtext)
    docs.append(text)
    
writeToDisk(docs)
    

In [28]:
import pandas as pd
import numpy as np

In [30]:
#2d array
size = 0
for i in range(len(docs)):
    for j in range (len(docs[i])):
        size +=1
        
size      

60977

In [33]:
cols = ["docID","text"]
df = pd.DataFrame(columns = cols)
# now fill it up row by row
idx = 0
for i in range(len(docs)):
        df.loc[idx] = [i,docs[i]]

        idx+=1
df.sort_values(["docID"], axis=0, ascending=True, inplace=True) 

df.head(10)

,docID,text
0,0,ABD'de işsizlik maaşı başvuruları geçen hafta ...
1,1,"Avrupa hisse senetleri, ABD endeks vadeli kont..."
2,2,"Enerji Piyasaları İşletme AŞ verilerine göre, ..."
3,3,Bazı Mallara Uygulanacak Özel Tüketim Vergisi ...
4,4,"İlk yarıda bankacılık endeksi yüzde 4,73, hold..."
5,5,ABD tahvil piyasalarında çalmaya başlayan rese...
6,6,Çin ABD'nin yüzde 10 yeni tarife ile Trump-Xi ...
7,7,Ekonomi basınında bugün Gazetelerin ekonomi sa...
8,8,İngiltere'de perakende satışlar Temmuz'da bek...
9,9,İhaleye 4 milyar 500 milyon liralık teklif gel...


In [34]:
from openpyxl import Workbook

In [35]:
text = ' '.join(df['text'])

In [36]:
#df.to_excel(join('/Users/cankutcoskun/Desktop/ChatBotProject', 'TurkishDataSet.xlsx'), index=False)